In [32]:
from pathlib import Path

examples_folder = Path('~/Projects/crosscompute-examples').expanduser()
automation_folder = configuration_folder = examples_folder / 'reports/randomize-histograms'
configuration_path = configuration_folder / 'automate.yml'
configuration_path

PosixPath('/home/rhh/Projects/crosscompute-examples/reports/randomize-histograms/automate.yml')

In [29]:
from crosscompute.routines.configuration import *

configuration = load_configuration(configuration_path)
raw_batch_definitions = configuration.get('batches', [])
raw_batch_definitions

[{'folder': 'batches/a'},
 {'folder': 'batches/me{mean}-va{variance}-vc{value_count}-bc{bin_count}',
  'configuration': {'path': 'datasets/batches.csv'}},
 {'folder': 'batches/me{mean}-va{variance}-vc{value_count}-bc{bin_count}',
  'reference': {'folder': 'batches/a'},
  'configuration': {'path': 'datasets/batches-vc-bc.csv'}}]

In [30]:
raw_batch_definition = raw_batch_definitions[0]
raw_batch_definition

{'folder': 'batches/a'}

In [31]:
if 'configuration' in raw_batch_definition:
    pass

In [ ]:
batch_folder = raw_batch_definition['folder']
absolute_batch_folder = join(automation_folder, batch_folder)
if exists(absolute_batch_folder):
    pass

In [ ]:
# If a configuration exists, load data_by_id from it
# If folder exists, load anything missing from it
# If reference exists, load remaining
# Check for missing variables
# Save extra variables in variables.dictionary

In [ ]:
# 1. reference folder
# 2. folder
# 3. configuration path

In [13]:
raw_batch_definition = raw_batch_definitions[0]
print(raw_batch_definition)
batch_definition = normalize_batch_definition(raw_batch_definition)
batch_slug = batch_definition['slug'] or format_slug(batch_definition['name'])                            
batch_definition['slug'] = batch_slug 
batch_definition['uri'] = BATCH_ROUTE.format(batch_slug=batch_slug) 
[batch_definition]

{'folder': 'batches/a'}


[{'folder': 'batches/a', 'name': 'a', 'slug': 'a', 'uri': '/b/a'}]

In [34]:
def get_batch_definitions_from_path(
        path, batch_definition, variable_definitions):
    file_extension = splitext(path)[1]
    try:
        yield_data_by_id = {
            '.csv': yield_data_by_id_from_csv,
            '.txt': yield_data_by_id_from_txt,
        }[file_extension]
    except KeyError:
        raise CrossComputeConfigurationError(
            f'{file_extension} not supported for batch configuration')
    batch_folder = batch_definition['folder']
    batch_name = batch_definition['name']
    batch_slug = batch_definition['slug']
    batch_definitions = []
    for data_by_id in yield_data_by_id(path, variable_definitions):
        folder = format_text(batch_folder, data_by_id)
        print('folder', folder)
        name = format_text(batch_name, data_by_id)
        slug = format_text(
            batch_slug, data_by_id) if batch_slug else format_slug(name)
        batch_definitions.append(batch_definition | {
            'folder': folder, 'name': name, 'slug': slug,
            'uri': BATCH_ROUTE.format(batch_slug=slug),
            'data_by_id': data_by_id})
    return batch_definitions

In [36]:
raw_batch_definition = raw_batch_definitions[1]
print(raw_batch_definition)
batch_definition = normalize_batch_definition(raw_batch_definition)
automation_folder = configuration['folder']                          
variable_definitions = get_variable_definitions(configuration, 'input') 
batch_configuration = raw_batch_definition['configuration']
get_batch_definitions_from_path(join(  
    automation_folder, batch_configuration['path'],  
), batch_definition, variable_definitions)

{'folder': 'batches/auto0/me{mean}-va{variance}-vc{value_count}-bc{bin_count}', 'configuration': {'path': 'datasets/batches.csv'}}
folder batches/auto0/me0-va1-vc1000-bc100
folder batches/auto0/me0-va1-vc3000-bc100
folder batches/auto0/me0-va1-vc5000-bc100


[{'folder': 'batches/auto0/me0-va1-vc1000-bc100',
  'name': 'me0-va1-vc1000-bc100',
  'slug': 'me0-va1-vc1000-bc100',
  'uri': '/b/me0-va1-vc1000-bc100',
  'data_by_id': {'mean': 0,
   'variance': 1,
   'value_count': 1000,
   'bin_count': 100}},
 {'folder': 'batches/auto0/me0-va1-vc3000-bc100',
  'name': 'me0-va1-vc3000-bc100',
  'slug': 'me0-va1-vc3000-bc100',
  'uri': '/b/me0-va1-vc3000-bc100',
  'data_by_id': {'mean': 0,
   'variance': 1,
   'value_count': 3000,
   'bin_count': 100}},
 {'folder': 'batches/auto0/me0-va1-vc5000-bc100',
  'name': 'me0-va1-vc5000-bc100',
  'slug': 'me0-va1-vc5000-bc100',
  'uri': '/b/me0-va1-vc5000-bc100',
  'data_by_id': {'mean': 0,
   'variance': 1,
   'value_count': 5000,
   'bin_count': 100}}]

In [32]:
len(raw_batch_definitions)

6

In [37]:
raw_batch_definition = raw_batch_definitions[2]
print(raw_batch_definition)
batch_definition = normalize_batch_definition(raw_batch_definition)

{'folder': 'batches/auto1/me{mean}-va{variance}-vc{value_count}-bc{bin_count}', 'variables': [{'id': 'mean', 'code': '[0, 10]'}, {'id': 'variance', 'code': 'range(1, 3)'}, {'id': 'value_count', 'code': "__import__('numpy').linspace(10, 10000, num=5)"}, {'id': 'bin_count', 'code': "__import__('numpy').logspace(1, 3, num=3)"}]}


In [15]:
'''
from crosscompute.routines.configuration import *

def get_batch_definitions(configuration):                                
    batch_definitions = []
    automation_folder = configuration['folder']                          
    variable_definitions = get_variable_definitions(                     
        configuration, 'input')                                          
    for raw_batch_definition in configuration.get('batches', []):        
        try:                                                             
            batch_definition = normalize_batch_definition(raw_batch_definition) 
            if 'configuration' in raw_batch_definition:                  
                batch_configuration = raw_batch_definition['configuration']
                if 'path' in batch_configuration:
                    definitions = get_batch_definitions_from_path(join(  
                        automation_folder, batch_configuration['path'],  
                    ), batch_definition, variable_definitions)
                # TODO: Support batch_configuration['uri']
                else:
                    raise CrossComputeConfigurationError(
                        'path expected for each batch configuration')    
            else:   
                batch_slug = batch_definition['slug'] or format_slug(    
                    batch_definition['name'])                            
                batch_definition['slug'] = batch_slug 
                batch_definition['uri'] = BATCH_ROUTE.format(            
                    batch_slug=batch_slug) 
                definitions = [batch_definition]
        except CrossComputeConfigurationError as e:                      
            L.error(e)
            continue
        batch_definitions.extend(definitions)                            
    return batch_definitions

get_batch_definitions(d)
''';